In [1]:
import tensorflow as tf
import random

In [2]:
( x_train, y_train ), ( x_test, y_test ) = tf.keras.datasets.mnist.load_data()

x_train = tf.cast( x_train / 255.0, tf.float32 )
x_test = tf.cast( x_test / 255.0, tf.float32 )

y_train = tf.keras.utils.to_categorical( y_train, 10 )
y_test = tf.keras.utils.to_categorical( y_test, 10 )
print( x_train.shape, x_test.shape )

(60000, 28, 28) (10000, 28, 28)


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
W1 = tf.Variable( tf.random.normal( [ 3, 3, 1, 32 ], stddev = 0.01 ) )
def Layer1( X ):
    l1 = tf.nn.conv2d( X, W1, strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
    l1 = tf.nn.relu( l1 )
    return ( tf.nn.max_pool( l1, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1 ], padding = 'SAME' ) )

W2 = tf.Variable( tf.random.normal( [ 3, 3, 32, 64 ], stddev = 0.01 ) )
def Layer2( X ):
    l2 = tf.nn.conv2d( Layer1( X ), W2, strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
    l2 = tf.nn.relu( l2 )
    return ( tf.nn.max_pool( l2, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1 ], padding = 'SAME' ) )

def Layer2_flat( X ):
    return ( tf.reshape( Layer2( X ), [ -1, 7 * 7 * 64 ] ) )

xavier = tf.keras.initializers.GlorotUniform()
W3 = tf.Variable( xavier( [ 7 * 7 * 64, 10 ] ) )
b = tf.Variable( tf.random.normal( [ 10 ] ) )

def Logits( X ):
    return ( tf.matmul( Layer2_flat( X ), W3 ) + b )

In [5]:
@tf.function
def Cost( X, Y ):
    return ( tf.reduce_mean( 
        tf.nn.softmax_cross_entropy_with_logits(
            logits = Logits( X ), labels = Y )
    ) )

def Minimize( X, Y ):
    loss = lambda: Cost( X, Y )
    
    tf.keras.optimizers.Adam( learning_rate ).minimize( loss, 
                                                      [ W1, W2, W3, b ] )

In [6]:
print( 'Learning started. It takes sometime' )
for epoch in range( training_epochs ):
    avg_cost = 0
    total_batch = int( len( x_train ) / batch_size )
    
    start_batch, end_batch = 0, batch_size
    for i in range( total_batch ):
        batch_xs, batch_ys = \
            x_train[ start_batch : end_batch ], y_train[start_batch : end_batch ]
        batch_xs = tf.reshape( batch_xs, [ -1, 28, 28, 1 ] )
        
        Minimize( batch_xs, batch_ys )
        cost_val = Cost( batch_xs, batch_ys )
        
        avg_cost += cost_val / total_batch
        
        start_batch = start_batch + batch_size
        end_batch = end_batch + batch_size
    
    print( 'Epoch: {:04d}, Cost: {:.9f}'.format( epoch + 1, avg_cost ) )
print( 'Learning Finished!' )

Learning started. It takes sometime
Epoch: 0001, Cost: 0.355469078
Epoch: 0002, Cost: 0.099221170
Epoch: 0003, Cost: 0.066990636
Epoch: 0004, Cost: 0.051718701
Epoch: 0005, Cost: 0.042779371
Epoch: 0006, Cost: 0.036943484
Epoch: 0007, Cost: 0.033430338
Epoch: 0008, Cost: 0.030721005
Epoch: 0009, Cost: 0.029415105
Epoch: 0010, Cost: 0.028434118
Epoch: 0011, Cost: 0.027779093
Epoch: 0012, Cost: 0.026592292
Epoch: 0013, Cost: 0.026220499
Epoch: 0014, Cost: 0.025310066
Epoch: 0015, Cost: 0.024163112
Learning Finished!


In [9]:
def CorrectPrediction( X, Y ):
    return ( tf.equal( tf.argmax( Logits( X ), 1 ), tf.argmax( Y, 1 ) ) )

def Accuracy( X, Y ):
    return ( tf.reduce_mean( tf.cast( CorrectPrediction( X, Y ), tf.float32 ) ) )

In [11]:
x_img = tf.reshape( x_test, [ -1, 28, 28, 1 ] )
tf.print( 'Accuracy: ', Accuracy( x_img, y_test ) )

Accuracy:  0.9867


In [13]:
r = random.randint( 0, len( x_test ) - 1 )
x_img = tf.reshape( x_test[ r: r + 1], [ -1, 28, 28, 1] )
tf.print( 'Label: ', tf.argmax( y_test[ r: r + 1 ], 1 ) )
tf.print( 'Prediction: ',
     tf.argmax( Logits( x_img ), 1 ) )

Label:  [4]
Prediction:  [4]
